# CEO Turnover Prediction using XGBoost
_**Using Gradient Boosted Trees to Predict the departures of CEOs**_

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Compile](#Compile)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
  1. [Relative cost of errors](#Relative-cost-of-errors)
1. [Next Steps](#Next-Steps)

---

## Background

_This notebook has been adapted from a [blog post](https://aws.amazon.com/blogs/ai/predicting-customer-churn-with-amazon-machine-learning/)_

In most companies, the top role is that of the Chief Executive Officer (CEO). The CEO role is a tough one to fill. In 2018, about 30% of the CEO departures in the Fortune 500 were involuntary, according to the Conference Board. Hiring and firing of a CEO is the responsibility of the board of a company, typically supported by a consultant. 

If the board could identify CEOs that are at risk of leaving early on, they could plan ahead. Stakeholders might care to know. A vacant CEO position is typically frowned upon by the investment community and viewed as a failure of the board to carry out on of its most important duties, that of appointing a CEO. 

This notebook is an experiment to use machine learning (ML) for the automated identification of CEO departures or attrition, also referred to in the academic literature as turnovers. ML models do not give perfect predictions, though, so this notebook is also about how to incorporate the relative costs of prediction mistakes when determining an outcome of using ML. 

For our model, we collected data on CEOs in North America from the past 20 years. While academic models make a distinction between forced departures and voluntary departures, this model does not follow that approach. For the purposes of this exercise, a turnover event is described as one taking place within the next 12 months. In other words, in the 11 months preceding the actual departure of the CEO, we should mark the turnover variable as **TRUE**.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

We start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
bucket = 'sagemaker-us-east-1-936165954724'
prefix = 'ml-turnover'

# Define IAM role
import awscli
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

Next, we'll import the Python libraries we'll need for the remainder of the exercise.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas.api.types as ptypes

import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

---
## Data

In this experiment, we are limited to public data (others m). Therefore, by design, it will be a relaaweak model.  

historical records on which customers ultimately ended up churning and which continued using the service. We can use this historical information to construct an ML model of one mobile operator’s churn using a process called training. After training the model, we can pass the profile information of an arbitrary customer (the same profile information that we used to train the model) to the model, and have the model predict whether this customer is going to churn. Of course, we expect the model to make mistakes–after all, predicting the future is tricky business! But I’ll also show how to deal with prediction errors.

The dataset we use is publicly available and was mentioned in the book [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) by Daniel T. Larose. It is attributed by the author to the University of California Irvine Repository of Machine Learning Datasets.  Let's download and read that dataset in now:

In [ ]:
!wget [TBD]
!unzip -o DKD2e_data_sets.zip

In [ ]:
df_ceos = pd.read_csv('./data/csv_files/ceos.csv.gz', compression='gzip')
pd.set_option('display.max_columns', 500)
df_ceos

In [ ]:
# Convert dataset to CSV file
df_ceos.to_csv("./data/csv_files/ceos.csv.gz", index=False)

By modern standards, it’s a relatively small dataset, with only 3,333 records, where each record uses 21 attributes to describe the profile of a customer of an unknown US mobile operator. The attributes are:

- `State`: the US state in which the customer resides, indicated by a two-letter abbreviation; for example, OH or NJ
- `Account Length`: the number of days that this account has been active
- `Area Code`: the three-digit area code of the corresponding customer’s phone number
- `Phone`: the remaining seven-digit phone number
- `Int’l Plan`: whether the customer has an international calling plan: yes/no
- `VMail Plan`: whether the customer has a voice mail feature: yes/no
- `VMail Message`: presumably the average number of voice mail messages per month
- `Day Mins`: the total number of calling minutes used during the day
- `Day Calls`: the total number of calls placed during the day
- `Day Charge`: the billed cost of daytime calls
- `Eve Mins, Eve Calls, Eve Charge`: the billed cost for calls placed during the evening
- `Night Mins`, `Night Calls`, `Night Charge`: the billed cost for calls placed during nighttime
- `Intl Mins`, `Intl Calls`, `Intl Charge`: the billed cost for international calls
- `CustServ Calls`: the number of calls placed to Customer Service
- `Churn?`: whether the customer left the service: true/false

The last attribute, `Churn?`, is known as the target attribute–the attribute that we want the ML model to predict.  Because the target attribute is binary, our model will be performing binary prediction, also known as binary classification.

Let's begin exploring the data:

In [42]:
# Get names of columns with missing values
def identify_missing(df):
    list = []
    # Ignore return columns (labeled 'rtn') for now 
    list = [col for col in df.columns if df[col].isnull().any()]
    return list

cols_with_missing = identify_missing(df_ceos)
cols_with_missing

['nationality',
 'network_size',
 'past_roles_tenure_avg',
 'company_roles_tenure_avg',
 'previous_ceo_tenure',
 'firm_rtn_1m',
 'firm_rtn_3m',
 'firm_rtn_6m',
 'firm_rtn_12m',
 'firm_rtn_24m',
 'firm_rtn_36m',
 'sic',
 'sector_rtn_1m',
 'sector_rtn_3m',
 'sector_rtn_6m',
 'sector_rtn_12m',
 'sector_rtn_24m',
 'sector_rtn_36m']

In [ ]:
X_train, X_valid, test_data = np.split(df_ceos.sample(frac=1, random_state=1024), [int(0.7 * len(df_ceos)), int(0.9 * len(df_ceos))])
X_train.to_csv('./data/training/train.csv', header=True, index=False)
X_valid.to_csv('./data//validation.csv', header=False, index=False)

In [ ]:
for col in cols_with_missing:
    df_ceos[col + '_was_missing'] = df_ceos[col].isnull()

### a) Imput missing NA values using a simple imputer (mean)

In [106]:

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

# Split the dataframe
def get_data_splits(df, test_fraction=0.15):
    """Splits a dataframe into train, validation, and test sets.

    First, orders by the column 'click_time'. Set the size of the 
    validation and test sets with the valid_fraction keyword argument.
    """

    df = df.sample(frac=1, random_state=1776)
    
    train_set, valid_set, test_set = np.array_split(df, [int((1-2*test_fraction)*len(df)), int((1-test_fraction)*len(df))])

    # a_train_set, a_test_set = train_test_split(a, train_size=0.85, test_fraction=0.15, random_state=10)
    # a_train_set, a_validation_set = train_test_split(a_train_set, train_size=0.85, test_fraction=0.15, random_state=10)
    # print(pd.concat([a_test_set, a_validation_set, a_train_set]))
    
    return train_set, valid_set, test_set

a_train_set, a_valid_set, a_test_set = get_data_splits(a)



,0,1,2,3,4
34,170,171,172,173,174
19,95,96,97,98,99
26,130,131,132,133,134
6,30,31,32,33,34
31,155,156,157,158,159
1,5,6,7,8,9


In [ ]:
##################################
#           COLUMNS BY TYPE      #
##################################

# Categorical features
cat_features = ['director_id', 'company_id',\
                'role_name','gender','nationality', \
                'role_id', 'sic']
                

# Drop columns
drop_cols = ['director_name', 'company_name','gvkey','tic'\
            'end_date', 'current_role', 'role_duration']

# Date features
date_features = ['start_date', 'date', 'date_of_birth']

# Timedelta features:
timedelta_features = ['company_roles_tenure','company_roles_tenure_avg','past_role_duration',\
                    'past_roles_tenure','past_roles_tenure_avg','previous_ceo_tenure',\
                    'role_tenure','tenure_at_ceo_start'
                    ]
# Numerical features
num_features = ['network_size','past_roles_count','age', \
                'company_roles_count','active_roles_count','active_roles_count_max',\
                'firm_rtn_1m', 'firm_rtn_3m', 'firm_rtn_6m',\
                'firm_rtn_12m', 'firm_rtn_24m', 'firm_rtn_36m',\
                'index_rtn_1m', 'index_rtn_3m', 'index_rtn_6m',\
                'index_rtn_12m', 'index_rtn_24m', 'index_rtn_36m',\
                'sector_rtn_1m', 'sector_rtn_3m', 'sector_rtn_6m',\
                'sector_rtn_12m','sector_rtn_24m', 'sector_rtn_36m'
                ]

# Boolean variables
bool_features = ['role_extension','nationality_was_missing',\
       'network_size_was_missing', 'past_roles_tenure_avg_was_missing',\
       'company_roles_tenure_avg_was_missing','ceo','chair', 'chair_ceo',\
       'previous_ceo_tenure_was_missing', 'firm_rtn_1m_was_missing',\
       'firm_rtn_3m_was_missing', 'firm_rtn_6m_was_missing',\
       'firm_rtn_12m_was_missing', 'firm_rtn_24m_was_missing',\
       'firm_rtn_36m_was_missing', 'sic_was_missing',\
       'sector_rtn_1m_was_missing', 'sector_rtn_3m_was_missing',\
       'sector_rtn_6m_was_missing', 'sector_rtn_12m_was_missing',\
       'sector_rtn_24m_was_missing', 'sector_rtn_36m_was_missing'
]

# Outcome variable
y_var = ['turnover']

In [9]:
# Convert time features to their corresponding data type (lost when saved as csv)

df_ceos[timedelta_features] = df_ceos[timedelta_features].apply(pd.to_timedelta)
df_ceos[cat_features] = df_ceos[cat_features].astype('object')
df_ceos[date_features] = df_ceos[date_features].apply(pd.to_datetime)

In [41]:
# Assert all data types are what they are supposed to be

def assert_col_types(df):
    print('Test column types:')
    try: 
        assert all(ptypes.is_object_dtype(df[col]) for col in cat_features)
        print(f'\n##### 1/5 Passed:\n')
        print(f'{df[cat_features].dtypes}')
        assert all(ptypes.is_timedelta64_dtype(df[col]) for col in timedelta_features)
        print(f'\n##### 2/5 Passed:\n')
        print(f'{df[timedelta_features].dtypes}')
        assert all(ptypes.is_datetime64_dtype(df[col]) for col in date_features)
        print(f'\n##### 3/5 Passed:\n')
        print(f'{df[date_features].dtypes}')
        assert all((ptypes.is_int64_dtype(df[col])|ptypes.is_float_dtype(df[col])) for col in num_features)
        print(f'\n##### 4/5 Passed:\n')
        print(f'{df[num_features].dtypes}')
        assert all(ptypes.is_bool_dtype(df[col]) for col in bool_features)
        print(f'\n##### 5/5 Passed:\n')
        print(f'{df[bool_features].dtypes}')
    except: 
        print('\nOne or more tests failed')
    
assert_col_types(df_ceos)

Test column types:

##### 1/5 Passed:

director_id    object
company_id     object
role_name      object
gender         object
nationality    object
role_id        object
sic            object
dtype: object

##### 2/5 Passed:

company_roles_tenure        timedelta64[ns]
company_roles_tenure_avg    timedelta64[ns]
past_role_duration          timedelta64[ns]
past_roles_tenure           timedelta64[ns]
past_roles_tenure_avg       timedelta64[ns]
previous_ceo_tenure         timedelta64[ns]
role_tenure                 timedelta64[ns]
tenure_at_ceo_start         timedelta64[ns]
dtype: object

##### 3/5 Passed:

start_date       datetime64[ns]
date             datetime64[ns]
date_of_birth    datetime64[ns]
dtype: object

##### 4/5 Passed:

network_size              float64
past_roles_count            int64
age                       float64
company_roles_count         int64
active_roles_count          int64
active_roles_count_max      int64
firm_rtn_1m               float64
firm_rtn_3m        

### Deal with some missing data

In [52]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

0.003556095692262982

In [48]:
# Split into training, validation, test sets
df_ceos.drop(drop_cols, axis=1, inplace=True)

numerical_missing_cols = ['firm_rtn_1m','firm_rtn_3m','firm_rtn_6m',\
            'firm_rtn_12m','firm_rtn_24m','firm_rtn_36m',\
            'sector_rtn_1m','sector_rtn_3m','sector_rtn_6m',\
            'sector_rtn_12m','sector_rtn_24m','sector_rtn_36m']

### Add some 'missing' and zeroes to certain columns with missing data 

df_ceos['nationality'].fillna("missing", inplace=True)
df_ceos['sic'].fillna("missing", inplace=True)
df_ceos['past_roles_tenure_avg'].fillna(0, inplace=True)
df_ceos['company_roles_tenure_avg'].fillna(0, inplace=True)


df_ceos['previous_ceo_tenure'].fillna(, inplace=True)
df_ceos['network_size'].fillna('mean', inplace=True)

1135515

In [ ]:
### 1. Apply one-hot encoding to cols with categorical data

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

###  Add the averages to the numerical cols

df['previous_ceo_tenure'].fillna(df['previous_ceo_tenure'].mean(), inplace=True)
df['previous_ceo_tenure'].fillna(df['previous_ceo_tenure'].mean(), inplace=True)
df['previous_ceo_tenure'].fillna(df['previous_ceo_tenure'].mean(), inplace=True)

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

In [44]:
# Convert DOB and start date to year and month
df_ceos = df_ceos.assign(dob_year =df_ceos.date_of_birth.dt.year,
               dob_month =df_ceos.date_of_birth.dt.month)

df_ceos = df_ceos.assign(start_date_year =df_ceos.start_date.dt.year,
               start_date_month =df_ceos.start_date.dt.month)

,date,director_name,company_name,role_name,director_id,company_id,date_of_birth,gender,nationality,network_size,role_id,current_role,role_duration,past_roles_count,company_roles_count,past_roles_tenure_avg,company_roles_tenure_avg,ceo,chair,chair_ceo,past_role_duration,past_roles_tenure,company_roles_tenure,active_roles_count,active_roles_count_max,previous_ceo_tenure,role_extension,end_date,start_date,role_tenure,turnover,gvkey,tic,firm_rtn_1m,firm_rtn_3m,firm_rtn_6m,firm_rtn_12m,firm_rtn_24m,firm_rtn_36m,index_rtn_1m,index_rtn_3m,index_rtn_6m,index_rtn_12m,index_rtn_24m,index_rtn_36m,age,tenure_at_ceo_start,sic,sector_rtn_1m,sector_rtn_3m,sector_rtn_6m,sector_rtn_12m,sector_rtn_24m,sector_rtn_36m,nationality_was_missing,network_size_was_missing,past_roles_tenure_avg_was_missing,company_roles_tenure_avg_was_missing,previous_ceo_tenure_was_missing,firm_rtn_1m_was_missing,firm_rtn_3m_was_missing,firm_rtn_6m_was_missing,firm_rtn_12m_was_missing,firm_rtn_24m_was_missing,firm_rtn_36m_was_missing,sic_was_missing,sector_rtn_1m_was_missing,sector_rtn_3m_was_missing,sector_rtn_6m_was_missing,sector_rtn_12m_was_missing,sector_rtn_24m_was_missing,sector_rtn_36m_was_missing,start_date_year,start_date_month
0,2018-09-30,A Beharelle,TRUEBLUE INC (Labor Ready Inc prior to 12/2007),President/CEO,1340849,18329,1969-01-01,M,NaN,2669.0,218,True,608 days 00:00:00.000000000,7,2,1381 days 20:34:17.142857,762 days 00:00:00,True,False,False,0 days,9673 days,1524 days,3,3,1206 days,False,2020-05-31,2018-09-30,0 days,False,22207.0,TBI,-0.110922,-0.033395,0.005792,0.160356,0.149603,0.159324,0.000250,0.066419,0.102813,0.154530,0.344012,0.513402,49.745032,1524 days,7363,-0.063492,0.050542,0.088124,0.245874,0.334664,0.460395,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2018,9
1,2018-09-30,Steve Cooper,TRUEBLUE INC (Labor Ready Inc prior to 12/2007),CEO,274662,18329,1962-01-01,M,American,721.0,5240630,False,1206 days 00:00:00.000000000,10,7,1229 days 14:24:00,1013 days 06:51:25.714285,True,False,False,1206 days,12296 days,7093 days,3,3,3284 days,False,2018-09-30,2015-05-31,1218 days,True,22207.0,TBI,-0.110922,-0.033395,0.005792,0.160356,0.149603,0.159324,0.000250,0.066419,0.102813,0.154530,0.344012,0.513402,56.745861,2603 days,7363,-0.063492,0.050542,0.088124,0.245874,0.334664,0.460395,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2015,5
2,2018-10-31,A Beharelle,TRUEBLUE INC (Labor Ready Inc prior to 12/2007),President/CEO,1340849,18329,1969-01-01,M,NaN,2669.0,218,True,608 days 00:00:00.000000000,7,2,1381 days 20:34:17.142857,762 days 00:00:00,True,False,False,0 days,9673 days,1524 days,3,3,1206 days,False,2020-05-31,2018-09-30,31 days,False,22207.0,TBI,-0.104415,-0.137523,-0.124578,-0.139114,0.333143,-0.194684,-0.074608,-0.043837,0.017711,0.046658,0.272652,0.299454,49.829908,1524 days,7363,-0.084437,-0.139710,0.017290,0.064980,0.300540,0.284561,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2018,9
3,2018-11-30,A Beharelle,TRUEBLUE INC (Labor Ready Inc prior to 12/2007),President/CEO,1340849,18329,1969-01-01,M,NaN,2669.0,218,True,608 days 00:00:00.000000000,7,2,1381 days 20:34:17.142857,762 days 00:00:00,True,False,False,0 days,9673 days,1524 days,3,3,1206 days,False,2020-05-31,2018-09-30,61 days,False,22207.0,TBI,0.082297,-0.138225,-0.021318,-0.112478,0.205251,-0.137931,0.017702,-0.057991,0.009514,0.036178,0.242836,0.318132,49.912045,1524 days,7363,-0.023455,-0.167763,-0.058713,-0.043130,0.155166,0.253644,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2018,9
4,2018-12-31,A Beharelle,TRUEBLUE INC (Labor Ready Inc prior to 12/2007),President/CEO,1340849,18329,1969-01-01,M,NaN,2669.0,218,True,608 days 00:00:00.000000000,7,2,1381 days 20:34:17.142857,762 days 00:00:00,True,False,False,0 days,9673 days,1524 days,3,3,1206

In [ ]:
# Remove rows with missing target, separate target from predictors
y = df_ceos['turnover']
df_ceos.drop(drop_cols, axis=1, inplace=True)
df_ceos.drop(['turnover'], axis=1, inplace=True)

# 




# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='mean')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
# ==========

# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()



from sklearn.preprocessing import OneHotEncoder


print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

### c) Use robust scalar to substract mean and divide by a variance measure for numerical cols

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

feature_cols = baseline_data.columns.drop('outcome')

# Keep 5 features
selector = SelectKBest(f_classif, k=5)

X_new = selector.fit_transform(baseline_data[feature_cols], baseline_data['outcome'])
X_new

In [ ]:
# Frequency tables for each categorical feature
for column in churn.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=churn[column], columns='% observations', normalize='columns'))

# Histograms for each numeric features
display(churn.describe())
%matplotlib inline
hist = churn.hist(bins=30, sharey=True, figsize=(10, 10))

We can see immediately that:
- `State` appears to be quite evenly distributed
- `Phone` takes on too many unique values to be of any practical use.  It's possible parsing out the prefix could have some value, but without more context on how these are allocated, we should avoid using it.
- Only 14% of customers churned, so there is some class imabalance, but nothing extreme.
- Most of the numeric features are surprisingly nicely distributed, with many showing bell-like gaussianity.  `VMail Message` being a notable exception (and `Area Code` showing up as a feature we should convert to non-numeric).

In [ ]:
churn = churn.drop('Phone', axis=1)
churn['Area Code'] = churn['Area Code'].astype(object)

Next let's look at the relationship between each of the features and our target variable.

In [ ]:
for column in churn.select_dtypes(include=['object']).columns:
    if column != 'Churn?':
        display(pd.crosstab(index=churn[column], columns=churn['Churn?'], normalize='columns'))

for column in churn.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = churn[[column, 'Churn?']].hist(by='Churn?', bins=30)
    plt.show()

Interestingly we see that churners appear:
- Fairly evenly distributed geographically
- More likely to have an international plan
- Less likely to have a voicemail plan
- To exhibit some bimodality in daily minutes (either higher or lower than the average for non-churners)
- To have a larger number of customer service calls (which makes sense as we'd expect customers who experience lots of problems may be more likely to churn)

In addition, we see that churners take on very similar distributions for features like `Day Mins` and `Day Charge`.  That's not surprising as we'd expect minutes spent talking to correlate with charges.  Let's dig deeper into the relationships between our features.

In [ ]:
display(churn.corr())
pd.plotting.scatter_matrix(churn, figsize=(12, 12))
plt.show()

We see several features that essentially have 100% correlation with one another.  Including these feature pairs in some machine learning algorithms can create catastrophic problems, while in others it will only introduce minor redundancy and bias.  Let's remove one feature from each of the highly correlated pairs: Day Charge from the pair with Day Mins, Night Charge from the pair with Night Mins, Intl Charge from the pair with Intl Mins:

In [ ]:
churn = churn.drop(['Day Charge', 'Eve Charge', 'Night Charge', 'Intl Charge'], axis=1)

Now that we've cleaned up our dataset, let's determine which algorithm to use.  As mentioned above, there appear to be some variables where both high and low (but not intermediate) values are predictive of churn.  In order to accommodate this in an algorithm like linear regression, we'd need to generate polynomial (or bucketed) terms.  Instead, let's attempt to model this problem using gradient boosted trees.  Amazon SageMaker provides an XGBoost container that we can use to train in a managed, distributed setting, and then host as a real-time prediction endpoint.  XGBoost uses gradient boosted trees which naturally account for non-linear relationships between features and the target variable, as well as accommodating complex interactions between features.

Amazon SageMaker XGBoost can train on data in either a CSV or LibSVM format.  For this example, we'll stick with CSV.  It should:
- Have the predictor variable in the first column
- Not have a header row

But first, let's convert our categorical features into numeric features.

In [ ]:
model_data = pd.get_dummies(churn)
model_data = pd.concat([model_data['Churn?_True.'], model_data.drop(['Churn?_False.', 'Churn?_True.'], axis=1)], axis=1)

And now let's split the data into training, validation, and test sets.  This will help prevent us from overfitting the model, and allow us to test the models accuracy on data it hasn't already seen.

In [ ]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

Now we'll upload these files to S3.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---
## Train

Moving onto training, first we'll need to specify the locations of the XGBoost algorithm containers.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')

Then, because we're training with the CSV file format, we'll create `s3_input`s that our training function can use as a pointer to the files in S3.

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

Now, we can specify a few parameters like what type of training instances we'd like to use and how many, as well as our XGBoost hyperparameters.  A few key hyperparameters are:
- `max_depth` controls how deep each tree within the algorithm can be built.  Deeper trees can lead to better fit, but are more computationally expensive and can lead to overfitting.  There is typically some trade-off in model performance that needs to be explored between a large number of shallow trees and a smaller number of deeper trees.
- `subsample` controls sampling of the training data.  This technique can help reduce overfitting, but setting it too low can also starve the model of data.
- `num_round` controls the number of boosting rounds.  This is essentially the subsequent models that are trained using the residuals of previous iterations.  Again, more rounds should produce a better fit on the training data, but can be computationally expensive or lead to overfitting.
- `eta` controls how aggressive each round of boosting is.  Larger values lead to more conservative boosting.
- `gamma` controls how aggressively trees are grown.  Larger values lead to more conservative models.

More detail on XGBoost's hyperparmeters can be found on their GitHub [page](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md).

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

---
## Compile
[Amazon SageMaker Neo](https://aws.amazon.com/sagemaker/neo/) optimizes models to run up to twice as fast, with no loss in accuracy. When calling `compile_model()` function, we specify the target instance family (m4) as well as the S3 bucket to which the compiled model would be stored.

In [ ]:
compiled_model = xgb
try:
    xgb.create_model()._neo_image_account(boto3.Session().region_name)
except:
    print('Neo is not currently supported in', boto3.Session().region_name)
else:
    output_path = '/'.join(xgb.output_path.split('/')[:-1])
    compiled_model = xgb.compile_model(target_instance_family='ml_m4', 
                                   input_shape={'data':[1, 69]},
                                   role=role,
                                   framework='xgboost',
                                   framework_version='0.7',
                                   output_path=output_path)
    compiled_model.name = 'deployed-xgboost-customer-churn'
    compiled_model.image = get_image_uri(sess.boto_region_name, 'xgboost-neo', repo_version='latest')

---
## Host

Now that we've trained the algorithm, let's create a model and deploy it to a hosted endpoint.

In [ ]:
xgb_predictor = compiled_model.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

### Evaluate

Now that we have a hosted endpoint running, we can make real-time predictions from our model very easily, simply by making an http POST request.  But first, we'll need to setup serializers and deserializers for passing our `test_data` NumPy arrays to the model behind the endpoint.

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

Now, we'll use a simple function to:
1. Loop over our test dataset
1. Split it into mini-batches of rows 
1. Convert those mini-batchs to CSV string payloads
1. Retrieve mini-batch predictions by invoking the XGBoost endpoint
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.as_matrix()[:, 1:])

There are many ways to compare the performance of a machine learning model, but let's start by simply by comparing actual to predicted values.  In this case, we're simply predicting whether the customer churned (`1`) or not (`0`), which produces a simple confusion matrix.

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.round(predictions), rownames=['actual'], colnames=['predictions'])

_Note, due to randomized elements of the algorithm, you results may differ slightly._

Of the 48 churners, we've correctly predicted 39 of them (true positives). And, we incorrectly predicted 4 customers would churn who then ended up not doing so (false positives).  There are also 9 customers who ended up churning, that we predicted would not (false negatives).

An important point here is that because of the `np.round()` function above we are using a simple threshold (or cutoff) of 0.5.  Our predictions from `xgboost` come out as continuous values between 0 and 1 and we force them into the binary classes that we began with.  However, because a customer that churns is expected to cost the company more than proactively trying to retain a customer who we think might churn, we should consider adjusting this cutoff.  That will almost certainly increase the number of false positives, but it can also be expected to increase the number of true positives and reduce the number of false negatives.

To get a rough intuition here, let's look at the continuous values of our predictions.

In [ ]:
plt.hist(predictions)
plt.show()

The continuous valued predictions coming from our model tend to skew toward 0 or 1, but there is sufficient mass between 0.1 and 0.9 that adjusting the cutoff should indeed shift a number of customers' predictions.  For example...

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.where(predictions > 0.3, 1, 0))

We can see that changing the cutoff from 0.5 to 0.3 results in 1 more true positives, 3 more false positives, and 1 fewer false negatives.  The numbers are small overall here, but that's 6-10% of customers overall that are shifting because of a change to the cutoff.  Was this the right decision?  We may end up retaining 3 extra customers, but we also unnecessarily incentivized 5 more customers who would have stayed.  Determining optimal cutoffs is a key step in properly applying machine learning in a real-world setting.  Let's discuss this more broadly and then apply a specific, hypothetical solution for our current problem.

### Relative cost of errors

Any practical binary classification problem is likely to produce a similarly sensitive cutoff. That by itself isn’t a problem. After all, if the scores for two classes are really easy to separate, the problem probably isn’t very hard to begin with and might even be solvable with simple rules instead of ML.

More important, if I put an ML model into production, there are costs associated with the model erroneously assigning false positives and false negatives. I also need to look at similar costs associated with correct predictions of true positives and true negatives.  Because the choice of the cutoff affects all four of these statistics, I need to consider the relative costs to the business for each of these four outcomes for each prediction.

#### Assigning costs

What are the costs for our problem of mobile operator churn? The costs, of course, depend on the specific actions that the business takes. Let's make some assumptions here.

First, assign the true negatives the cost of \$0. Our model essentially correctly identified a happy customer in this case, and we don’t need to do anything.

False negatives are the most problematic, because they incorrectly predict that a churning customer will stay. We lose the customer and will have to pay all the costs of acquiring a replacement customer, including foregone revenue, advertising costs, administrative costs, point of sale costs, and likely a phone hardware subsidy. A quick search on the Internet reveals that such costs typically run in the hundreds of dollars so, for the purposes of this example, let's assume \$500. This is the cost of false negatives.

Finally, for customers that our model identifies as churning, let's assume a retention incentive in the amount of \$100. If my provider offered me such a concession, I’d certainly think twice before leaving. This is the cost of both true positive and false positive outcomes. In the case of false positives (the customer is happy, but the model mistakenly predicted churn), we will “waste” the \$100 concession. We probably could have spent that \$100 more effectively, but it's possible we increased the loyalty of an already loyal customer, so that’s not so bad.

#### Finding the optimal cutoff

It’s clear that false negatives are substantially more costly than false positives. Instead of optimizing for error based on the number of customers, we should be minimizing a cost function that looks like this:

```txt
$500 * FN(C) + $0 * TN(C) + $100 * FP(C) + $100 * TP(C)
```

FN(C) means that the false negative percentage is a function of the cutoff, C, and similar for TN, FP, and TP.  We need to find the cutoff, C, where the result of the expression is smallest.

A straightforward way to do this, is to simply run a simulation over a large number of possible cutoffs.  We test 100 possible values in the for loop below.

In [ ]:
cutoffs = np.arange(0.01, 1, 0.01)
costs = []
for c in cutoffs:
    costs.append(np.sum(np.sum(np.array([[0, 100], [500, 100]]) * 
                               pd.crosstab(index=test_data.iloc[:, 0], 
                                           columns=np.where(predictions > c, 1, 0)))))

costs = np.array(costs)
plt.plot(cutoffs, costs)
plt.show()
print('Cost is minimized near a cutoff of:', cutoffs[np.argmin(costs)], 'for a cost of:', np.min(costs))

The above chart shows how picking a threshold too low results in costs skyrocketing as all customers are given a retention incentive.  Meanwhile, setting the threshold too high results in too many lost customers, which ultimately grows to be nearly as costly.  The overall cost can be minimized at \$8400 by setting the cutoff to 0.46, which is substantially better than the \$20k+ I would expect to lose by not taking any action.

---
## Extensions

This notebook showcased how to build a model that predicts whether a customer is likely to churn, and then how to optimally set a threshold that accounts for the cost of true positives, false positives, and false negatives.  There are several means of extending it including:
- Some customers who receive retention incentives will still churn.  Including a probability of churning despite receiving an incentive in our cost function would provide a better ROI on our retention programs.
- Customers who switch to a lower-priced plan or who deactivate a paid feature represent different kinds of churn that could be modeled separately.
- Modeling the evolution of customer behavior. If usage is dropping and the number of calls placed to Customer Service is increasing, you are more likely to experience churn then if the trend is the opposite. A customer profile should incorporate behavior trends.
- Actual training data and monetary cost assignments could be more complex.
- Multiple models for each type of churn could be needed.

Regardless of additional complexity, similar principles described in this notebook are likely apply.

### (Optional) Clean-up

If you're ready to be done with this notebook, please run the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)